## Week 3
Using Ciera Martinez's map_motif.py in a generalized jupyter notebook function.

Author: Jemima Shi

In [10]:
import numpy as np
import pandas as pd
import re
import Bio
from Bio import SeqIO
from glob import glob
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

import os, sys
from Bio import motifs
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC, generic_dna, generic_protein
from collections import defaultdict

### Data Processing

Reads in the alligned raw DNA sequences and motifs to be pipelined:

1. Saves a copy of the sequence without the '$-$' included.

2. Casts the undashed version as IUPACUnambiguousDNA().

3. Calculates the thresholds to be used in assigning scores for likihood of the motif appearing.

4. Takes the indicies that have been scored and maps them to the corresponding raw sequence index, taking into consideration if the strand was positive or negative.

5. Returns a df with $\scriptsize{[\textbf{position}, \textbf{score, sequence length}, \textbf{species}, \textbf{raw_position index}, \textbf{strand positioning}, \textbf{aligned position}]}$.

*caluclate_all_TFBS(files, all_motifs) will start the pipeline

In [11]:
#Reads through and saves the names of all same files(.fa and .fm respectively) found in the same directory.

files = glob('*.fa')
all_motifs = glob('*.fm')

In [12]:

def read_records(file):
    """
    file: a single .fa file within the directory.
    
    Returns the records from a .fa file as a list .
    """
    return list(SeqIO.parse(file, "fasta"))

In [13]:

def read_motif(motif):
    """
    motif: a single .fm file within the directory.
    
    Returns a motif.jaspar.Motif (the motif matrix) from the .fm file.
    """
    return motifs.read(open(motif), "pfm")

In [14]:
first_file = read_records(files[0])
first_motif = read_motif(all_motifs[0])

In [15]:

def raw_sequence_ungap(records):
    """
    records: a list of species within a single .fa file.
    
    Returns a list of Sequences with an ungapped('-') sequence.
    """
    raw_sequences = []
    for record in records:
        raw_sequences.append(SeqRecord(record.seq.ungap("-"), id = record.id))
    return raw_sequences

In [16]:

def cast_sequence(ungapped_sequence):
    """
    ungapped_sequence: a list with the sequence and id for all the species in a file.
    
    Returns a list sequences with the type cast as IUPACUnambiguousDNA().
    """
    casted = []
    for record in ungapped_sequence:
        casted.append(Seq(str(record.seq), IUPAC.IUPACUnambiguousDNA()))
    return casted

In [17]:
first_raw_ungapped = raw_sequence_ungap(first_file)

In [18]:
first_cast = cast_sequence(first_raw_ungapped)

In [19]:

def calculate_pssm(chosen_motif, pseudocounts=0.0):
    """
    chosen_motif: Chosen motif sequence to be used in creating the pwm
    pseudocounts: Default set to 0.0, otherwise added to the counts before calculating position-weight matrix.
    
    Returns the pssm (PositionSpecificScoringMatrix)
    """
    pwm = chosen_motif.counts.normalize(pseudocounts)
    return pwm.log_odds()

In [20]:

def pwm_threshold(cast_sequences):
    """
    cast_sequences: A list of sequences that have been ungapped and casted.
    
    Returns a list of pssm values
    """
    pssm = calculate_pssm(raw_sequences)
    pssm_list = [ ]
    for seq in cast_sequences:
        pssm_list.append(pssm.calculate(seq))
    return pssm_list

In [21]:

def extract_len_id(raw_sequences):
    """
    raw_sequences: A list of raw sequences that hasn't yet cast.
    
    Returns a list of dictionary pairs for the species and seq_len of each sequence.
    """
    raw_id = []
    record_length = []
    for seq in raw_sequences:
        try:
            raw_id.append(seq.id)
            record_length.append(len(seq))
        except AttributeError:
            raw_id.append('n/a')
            record_length.append(len(seq))
    return [{'species': x, 'seq_len': y} for x, y in zip(raw_id, record_length)]

In [22]:

def positions(raw_sequence, cast_sequences, motif, chosen_precision=10**4):
    """
    raw_sequence: A list of the sequences that have been ungapped but not casted.
    cast_sequences: A list of sequences that have been ungapped and casted.
    motif: the chosen motif file read in using read_motif
    precision: default precision set to 10**4 to be used in the pssm distribution.
    
    Returns a list of positions
    """
    pssm = calculate_pssm(motif)
    distribution = pssm.distribution(background=motif.background, precision= chosen_precision)
    patser_threshold = distribution.threshold_patser()
    
    position_list = []
    len_and_ids = extract_len_id(raw_sequence)
    for i in range(0,8): #number of gene sequences 
        for position, score in pssm.search(cast_sequences[i], patser_threshold):
            positions = {'species': len_and_ids[i].get('species'), 'score':score, 
                         'position':position, 'seq_len': len_and_ids[i].get('seq_len') }
            position_list.append(positions)
    return position_list

In [23]:

def positive_positions(df):
    """
    df: The df with position, score, seq_len, and species.
    
    Returns a df where the position arguments are translated into positive ints.
    """
    temp_pos = df[df["position"] >= 0].copy()
    temp_neg = df[df["position"] < 0].copy()
    temp_pos["raw_position"] = temp_pos["position"]
    temp_neg["raw_position"] = temp_neg["seq_len"] + temp_neg["position"]  
    temp_together = temp_pos.append(temp_neg).reset_index().sort_values("index")

    return temp_together.set_index("index")

In [24]:

def define_sign(df):
    """
    df: The df with position, score, seq_len, species, and raw_position.
    
    Returns the df with a strand sign column appended on.
    """
    df["strand"] = np.where(df['position'] >= 0, 'positive', 'negative')
    return df

In [25]:
test_df = pd.DataFrame(positions(first_raw_ungapped, first_cast, first_motif))
t = positive_positions(test_df)
t = define_sign(t)

In [26]:

def merge_align_df(raw_df, aligned_seq):
    """
    raw_df: A pandas df that contains the positions, score, seq_len, species, and strand orientation.
    aligned_seq: Original inputted sequence with the '-' still included.
    
    Returns a df with alignned index appended to raw_df sorted by ['species', 'raw_position'] 
    with N/A values dropped.
    
    """
    remap_list = []
    nuc_list = ['A', 'a', 'G', 'g', 'C', 'c', 'T', 't', 'N', 'n']

    for i in range(0,9):
        counter = 0
        for xInd, x in enumerate(aligned_seq[i].seq):    
            if x in nuc_list:
                remaps = {'raw_position': counter, 'align_position':xInd, 'species':aligned_seq[i].id}
                counter += 1
                remap_list.append(remaps)
            
    remap_DF = pd.DataFrame(remap_list)
    TFBS_map_DF_all = pd.merge(raw_df, remap_DF, on=['species', 'raw_position'], how='outer')
    TFBS_map_DF_all = TFBS_map_DF_all.sort_values(by=['species','align_position'], ascending=[True, True])

    return TFBS_map_DF_all.dropna()   

In [47]:
temp = merge_align_df(t, first_file)

In [28]:

def save_df(TFBS_df, align_file, motif_file):
    """
    TFBS_df: A pandas df with the raw/aligned position, score, and file names.
    align_file: Name of read in alignment file.
    motif_file: Name of read in motif file.
    
    Returns a csv copy of the df stored in the current directory
    """
    return TFBS_df.to_csv('map_motif ' + align_file + "-" + motif_file + ".csv", sep='\t', na_rep="NA")

In [29]:

def caluclate_all_TFBS(files, all_motifs):
    """
    files: A list of all the .fa files in the current directory.
    all_motifs: A list of all the .fm files in the current directory.
    
    Returns a csv with the positions, score, and orientation saved into the current directory for all files.
    """
    for file in files:
        for motif in all_motifs:
            curr_file = read_records(file)
            curr_motif = read_motif(motif)
            curr_raw = raw_sequence_ungap(curr_file)
            curr_cast = cast_sequence(curr_raw)
            
            raw_df = pd.DataFrame(positions(curr_raw, curr_cast, curr_motif))
            temp_df = positive_positions(raw_df)
            temp_df = define_sign(temp_df)
            
            align_name = re.split(r'_', files[0])[-1]
            save_df(merge_align_df(temp_df, curr_file), align_name, motif)

In [31]:
#Run this cell to generate a csv file for all the files with all the possible motif files.

caluclate_all_TFBS(files, all_motifs)

#### Example df output for the chosen .fa (aligned DNA sequence) file and .fm (motif probabilities) file.

$\scriptsize\textbf{Fasta}$: 'align_outlier_rm_with_length_VT0809.fa'

$\scriptsize\textbf{Motif}$: 'bcd_cell2008.fm'

In [49]:
temp

,position,score,seq_len,species,raw_position,strand,align_position
6671,-2381.0,0.654739,2381.0,VT0809|1|MEMB002A|-|2381,0,negative,0
6672,2.0,1.590609,2381.0,VT0809|1|MEMB002A|-|2381,2,positive,2
6673,7.0,4.699133,2381.0,VT0809|1|MEMB002A|-|2381,7,positive,7
6674,11.0,0.368217,2381.0,VT0809|1|MEMB002A|-|2381,11,positive,11
6675,12.0,2.529208,2381.0,VT0809|1|MEMB002A|-|2381,12,positive,12
6676,-2365.0,1.462094,2381.0,VT0809|1|MEMB002A|-|2381,16,negative,16
6677,18.0,2.982926,2381.0,VT0809|1|MEMB002A|-|2381,18,positive,18
6678,19.0,1.529209,2381.0,VT0809|1|MEMB002A|-|2381,19,positive,19
6679,-2358.0,2.114171,2381.0,VT0809|1|MEMB002A|-|2381,23,negative,23
6680,24.0,4.699133,2381.0,VT0809|1|MEMB002A|-|2381,24,positive,24


### Data Extraction

Taking the dfs that come from the "caluclate_all_TFBS" function to locate and then save the given DNA sequence with '-' included in both the sequence and indices.

In [49]:
def filter_95_percentile(TFBS_df):
    """
    TFBS_df: A pandas df with the raw/aligned position, score, and file names.
    
    returns the TFBS_df with only values above the 95th percentile.
    """
    mean = np.mean(TFBS_df["score"])
    sds = 2*np.std(TFBS_df["score"])
    filtered_df = TFBS_df[TFBS_df["score"] >= mean + sds]
    return filtered_df

In [234]:
def map_raw_seq(TFBS_df, seq_file):
    """
    TFBS_df: The TFBS_df with entries only above the 95th percentile.
    seq_file: a single .fa file within the directory.
    
    returns a df with scores in the upper 5th percentile along with the species'
    corresponding raw sequence.
    """
    
    values = [[record.seq, record.description] for record in seq_file]
    sequences = pd.DataFrame(values)
    sequences.rename(columns={0 : 'Sequence', 1: 'Description'}, inplace=True)
    sequences = sequences.set_index("Description")
    mapping = sequences.to_dict("index")
    
    upper_fifth = filter_95_percentile(TFBS_df)
    upper_fifth = upper_fifth[["species", "raw_position", "strand"]]
    upper_fifth["sequence"] = upper_fifth["species"].apply(lambda x: str(mapping.get(x).get("Sequence")))
    
    upper_fifth = upper_fifth.reset_index().drop("index", axis=1)
    
    return upper_fifth

In [237]:
matched = map_raw_seq(temp, first_file)
matched

,species,raw_position,strand,sequence
0,VT0809|1|MEMB002A|-|2381,7,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
1,VT0809|1|MEMB002A|-|2381,24,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
2,VT0809|1|MEMB002A|-|2381,114,negative,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
3,VT0809|1|MEMB002A|-|2381,118,negative,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
4,VT0809|1|MEMB002A|-|2381,222,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
5,VT0809|1|MEMB002A|-|2381,225,negative,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
6,VT0809|1|MEMB002A|-|2381,231,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
7,VT0809|1|MEMB002A|-|2381,263,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
8,VT0809|1|MEMB002A|-|2381,283,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...
9,VT0809|1|MEMB002A|-|2381,410,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...


In [267]:
def positive_index_seq(matched_df, motif):
    """
    matched_df: df with the species matched with their corresponding raw sequence.
    motif: a single .fm file within the directory.
    
    returns the df with appended columns of the DNA sequences/indices starting from the 
    raw_index from positively oriented scores and then going on until the proper number
    of letters have been found.
    """
    motif = str(first_motif.consensus)
    motif_len = len(motif)
    
    extracted_seqs = []
    extracted_indices = []
    
    pos = matched_df[matched_df["strand"] == "positive"]
    final = pos.copy()
    for i in np.arange(pos.shape[0]):
        curr_row = pos.iloc[i]
        curr_seq = curr_row["sequence"]
        index = curr_row["raw_position"]
        
        selected_seq = curr_seq[index:index + motif_len]     
        letters = re.findall(r'[^-]', selected_seq)
        additional_indicies = 0
        
        if len(letters) != motif_len: 
            while len(re.findall(r'[^-]', selected_seq)) != motif_len:
                additional_indicies += 1
                selected_seq = curr_seq[index:index + motif_len + additional_indicies]
                 
        selected_index = "{0}:{1}".format(index, index + motif_len + additional_indicies)
        extracted_seqs.append(selected_seq)
        extracted_indices.append(selected_index)
        
    final["extracted_seq"] = extracted_seqs
    final["extracted_indices"] = extracted_indices
    return final

In [256]:
extracted = positive_index_seq(matched, first_motif)
extracted

,species,raw_position,strand,sequence,extracted_seq
0,VT0809|1|MEMB002A|-|2381,7,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TCAG
1,VT0809|1|MEMB002A|-|2381,24,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TCAG
4,VT0809|1|MEMB002A|-|2381,222,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CTTG
6,VT0809|1|MEMB002A|-|2381,231,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,ACGC
7,VT0809|1|MEMB002A|-|2381,263,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,GGCT
8,VT0809|1|MEMB002A|-|2381,283,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CT--
9,VT0809|1|MEMB002A|-|2381,410,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,----
10,VT0809|1|MEMB002A|-|2381,459,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CTGC
11,VT0809|1|MEMB002A|-|2381,476,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,T-TG
13,VT0809|1|MEMB002A|-|2381,581,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TTTG


In [263]:
positive_index_seq(matched, first_motif)

,species,raw_position,strand,sequence,extracted_seq,extracted_indices
0,VT0809|1|MEMB002A|-|2381,7,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TCAG,7:11
1,VT0809|1|MEMB002A|-|2381,24,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TCAG,24:28
4,VT0809|1|MEMB002A|-|2381,222,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CTTG,222:226
6,VT0809|1|MEMB002A|-|2381,231,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,ACGC,231:235
7,VT0809|1|MEMB002A|-|2381,263,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,GGCT,263:267
8,VT0809|1|MEMB002A|-|2381,283,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CT-----CA,283:292
9,VT0809|1|MEMB002A|-|2381,410,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,-------GCCA,410:421
10,VT0809|1|MEMB002A|-|2381,459,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,CTGC,459:463
11,VT0809|1|MEMB002A|-|2381,476,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,T-TGC,476:481
13,VT0809|1|MEMB002A|-|2381,581,positive,CGCAACATCAGCCAAGCATAAAGCTCAGCCCCATCGCACA--CCAG...,TTTG,581:585


In [277]:
def find_motif_pipeline(TFBS_df, seq_file, motif):
    """
    TFBS_df: A pandas df with the raw/aligned position, score, and file names.
    seq_file: a single .fa file within the directory.
    motif: a single .fm file within the directory.
    
    returns the execution of the pipeline for filtering out the calculated motif locations
    and the actual DNA sequence.
    """
    matched = map_raw_seq(TFBS_df, seq_file)
    extracted = positive_index_seq(matched, motif)
    final = positive_index_seq(matched, motif)
    return final[["species", "raw_position", "strand", "extracted_seq", "extracted_indices"]]


In [278]:
find_motif_pipeline(temp, first_file, first_motif)

,species,raw_position,strand,extracted_seq,extracted_indices
0,VT0809|1|MEMB002A|-|2381,7,positive,TCAG,7:11
1,VT0809|1|MEMB002A|-|2381,24,positive,TCAG,24:28
4,VT0809|1|MEMB002A|-|2381,222,positive,CTTG,222:226
6,VT0809|1|MEMB002A|-|2381,231,positive,ACGC,231:235
7,VT0809|1|MEMB002A|-|2381,263,positive,GGCT,263:267
8,VT0809|1|MEMB002A|-|2381,283,positive,CT-----CA,283:292
9,VT0809|1|MEMB002A|-|2381,410,positive,-------GCCA,410:421
10,VT0809|1|MEMB002A|-|2381,459,positive,CTGC,459:463
11,VT0809|1|MEMB002A|-|2381,476,positive,T-TGC,476:481
13,VT0809|1|MEMB002A|-|2381,581,positive,TTTG,581:585


Example output for positively oriented DNA sequences when trying to locate the motif. 